# Problem 3
Authors: Sid Murching, Suraj Nair, Alex Cui

In [1]:
import numpy as np
from P3CHelpers import *
from keras.models import Sequential
from keras.layers import Dense, Activation
import sys


%load_ext blackcellmagic

Using TensorFlow backend.


## 3D:
Fill in the generate_traindata and find_most_similar_pairs functions

In [2]:
def get_word_repr(word_to_index, word):
    """
    Returns one-hot-encoded feature representation of the specified word given
    a dictionary mapping words to their one-hot-encoded index.

    Arguments:
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        word:          String containing word whose feature representation we wish to compute.

    Returns:
        feature_representation:     Feature representation of the passed-in word.
    """
    unique_words = word_to_index.keys()

    # Return a vector that's zero everywhere besides the index corresponding to <word>

    feature_representation = np.zeros(len(unique_words))
    feature_representation[word_to_index[word]] = 1
    return feature_representation


def generate_traindata(word_list, word_to_index, window_size=4):
    """
    Generates training data for Skipgram model.

    Arguments:
        word_list:     Sequential list of words (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window.
                       (use the default value when running your code).

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training 
                              points (one-hot-encoded vectors representing individual words) and 
                              their corresponding labels (also one-hot-encoded vectors representing words).

                              For each index i, trainX[i] should correspond to a word in
                              <word_list>, and trainY[i] should correspond to one of the words within
                              a window of size <window_size> of trainX[i].
    """
    trainX = []
    trainY = []

    for word in word_list:
        
        index = word_to_index[word]

        for i in [x for x in range(-window_size, window_size + 1) if x != 0]:

            if (index + i) >= 0 and (index + i) <= len(word_list):

                onehot_x = get_word_repr(word_to_index, word_list[index])
                onehot_y = get_word_repr(word_to_index, word_list[index + i])

                trainX.append(onehot_x)
                trainY.append(onehot_y)

    return (np.array(trainX), np.array(trainY))

In [3]:
"""Keras version."""

def find_most_similar_pairs(filename, num_latent_factors):
    """
    Find the most similar pairs from the word embeddings computed from
    a body of text
    
    Arguments:
        filename:           Text file to read and train embeddings from
        num_latent_factors: The number of latent factors / the size of the embedding
    """
    # Load in a list of words from the specified file; remove non-alphanumeric characters
    # and make all chars lowercase.
    sample_text = load_word_list(filename)

    # Create dictionary mapping unique words to their one-hot-encoded index
    word_to_index = generate_onehot_dict(sample_text)

    # Create training data using default window size
    trainX, trainY = generate_traindata(sample_text, word_to_index)

    # TODO: 1) Create and train model in Keras.

    # vocab_size = number of unique words in our text file. Will be useful when adding layers
    # to your neural network
    vocab_size = len(word_to_index)
    
    model = Sequential()

    model.add(Dense(num_latent_factors, input_dim=vocab_size))
    model.add(Dense(vocab_size))
    model.add(Activation("softmax"))

    model.compile(
        loss="categorical_crossentropy", 
        optimizer="Adam", 
        metrics=["accuracy"]
    )
    
    print(model.summary())
    print("\n")
        
    model.fit(trainX, trainY, batch_size=30, epochs=80)

    # TODO: 2) Extract weights for hidden layer, set <weights> variable below

    weights = model.get_weights()[0]
    
    print("\n")

    for weight in model.get_weights():
        print("Weight Shape:", weight.shape)

    # Find and print most similar pairs
    similar_pairs = most_similar_pairs(weights, word_to_index)
    print("\n")
    print("Most Similar Pairs:")
    for pair in similar_pairs[:30]:
        print(pair)

## 3G:
Run the function below and report your results for dr_seuss.txt.

In [ ]:
find_most_similar_pairs('syke.txt', 10)